# Homework 4: Dimension Reduction

In this homework, we are analyzing Fashion-MNIST, an image dataset for the fashion recognition. However, we do not involve with the any recognition task, but focus on PCA and T-SNE.
The dataset is also available on Kaggle,
https://www.kaggle.com/datasets/zalando-research/fashionmnist?resource=download

Since the dataset is huge, we only use the first 1000 data points from the dataset. Each row in the dataset represents a grayscale image of 28x28. So each row has 28x28 dimension for the grayscale value. Also, the dataset has a label column to indicate what the image is about. The index value of the label is as followed,

0 T-shirt/top
1 Trouser
2 Pullover
3 Dress
4 Coat
5 Sandal
6 Shirt
7 Sneaker
8 Bag
9 Ankle boot

## Part 1: Dataset Loading

Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.manifold import (
    TSNE,
)
from sklearn.decomposition import PCA
import sklearn
from sklearn.datasets import load_linnerud

from PIL import Image

### Loading dataset with pandas (1 point)

In [2]:
data = pd.read_csv("./fashion-mnist.csv")
# data = data[:1000]

The expected result is followed,

In [3]:
# data.to_csv("./fashion-mnist.csv", index=False)
data

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9,0,0,0,0,0,0,0,0,0,...,0,2,0,40,143,103,103,98,122,37
996,2,0,0,0,0,0,0,0,2,0,...,0,0,0,0,60,156,29,0,0,0
997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,4,0,0,0,0,0,0,0,1,1,...,0,0,0,46,126,105,81,0,0,0


Split the labels (y) and the image values (X)

hints:
+ select the subset of the pandas dataframe: subset = data[["col1", "col2"]]
+ get all the column names of the dataframe: columns = list(data.columns)

In [5]:
X = data[filter(lambda x: x!= "label", list(data.columns))]
y = data["label"]

The expected results should be:

In [6]:
X.shape

(1000, 784)

In [7]:
y.shape

(1000,)

## Part 2: Helper Functions (2 points)

Imlement the helper function to draw the figure


In [149]:
import plotly.express as px  # extra graphing package
target_key = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
key_dict = {}  # just another format of target_key
for i in range(len(target_key)):
    key_dict[i] = target_key[i]
# print(px.data.iris())

def plot_clustering(X_red, labels, title="No Title", model_type="Cluster"):  # TODO
    for n in [3]: # [2,3]:
        for discrete in [True, False]:
            if model_type == "Cluster":
                model = PCA(n_components=n)
                model.fit(X_red)
                X_new = model.transform(X_red)
                col_labels = [f"PCA {i+1} ({var:.1f}%)" for i, var in enumerate(model.explained_variance_ratio_ * 100)]
            elif model_type == "TSNE":
                X_new = sklearn.manifold.TSNE(n_components=n, learning_rate='auto', init='random').fit_transform(X_red)
                # Note: experimentation with perplixity yields similar results (perplexity=(X_red.shape[0]/len(target_key)))
                col_labels = [model_type+" "+str(i) for i in range(X_new.shape[1])]
            else:
                print("ERROR")
                return
            
            if discrete:
                temp_labels = np.array([target_key[val] for val in labels.values]).reshape(-1, 1)
            else:
                temp_labels = labels.values.reshape(-1,1)
                print("Key: ", key_dict)
            temp_data = np.hstack((X_new, temp_labels))
        #     print(col_labels)
            df = pd.DataFrame(data=temp_data, columns=col_labels+["clothing_type"])
        #     print(df)
            print("Data after dimensionality reduction:\n", df)

            fig = px.scatter_matrix(
                df,
                dimensions=col_labels,
                color="clothing_type"
            )
            fig.update_traces(diagonal_visible=False)
            print("\n - -",title, "for", n, ("Discrete" if discrete else "Continuous"), "Clusters:")
            fig.show()
            print('\n\n')

In [140]:
def show_X(tempX):  # X.values.tolist()[ind]
    temp = popup(tempX)
    img = Image.fromarray(temp)
    img.show()
    
def popup(data):
    num = int(np.sqrt(len(data)))
    ret = np.zeros((num,num))
    for i in range(num*num):
        b = i//num
        a = i%num
        ret[a, b] = data[i]
    return ret

## Part 3: PCA (2 points)
Please use the sklearn's PCA class to transform the original data

In [135]:
X_pca = X
plot_clustering(X_pca, y, "PCA", "Cluster")

Data after dimensionality reduction:
            PCA 1 (28.7%)        PCA 2 (18.1%)         PCA 3 (6.4%)  \
0     1369.2755128680244  -412.75494116058735   188.58003219691852   
1      -70.8611898555389  -1579.4851409070286    271.7934500982604   
2     -792.5036134192308  -136.18444679166262   -922.3094478153929   
3      833.7050169952452    190.1903643922748   -735.1711698455306   
4     240.20956436791195   -983.2030233527606    105.4068082850465   
..                   ...                  ...                  ...   
995     697.263658732013    598.7755701624557   1234.8930225592708   
996    2085.808791556092     802.723667347248   -353.0678865516996   
997  -1257.6550431513149   469.94981718879995    323.5115725527468   
998   1714.9190789913405   1002.4800279346267  -506.94507459504257   
999  -1707.8172418901872   -129.6173123274609   -618.1300902549656   

    clothing_type  
0     T-shirt/top  
1         Trouser  
2        Pullover  
3        Pullover  
4           Dress  
.




Key:  {0: 'T-shirt/top', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat', 5: 'Sandal', 6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle boot'}
Data after dimensionality reduction:
      PCA 1 (28.7%)  PCA 2 (18.1%)  PCA 3 (6.4%)  clothing_type
0      1369.275513    -412.754941    188.580044            0.0
1       -70.861190   -1579.485141    271.793428            1.0
2      -792.503613    -136.184447   -922.309458            2.0
3       833.705017     190.190364   -735.171244            2.0
4       240.209564    -983.203023    105.406803            3.0
..             ...            ...           ...            ...
995     697.263659     598.775570   1234.892946            9.0
996    2085.808792     802.723667   -353.067885            2.0
997   -1257.655043     469.949817    323.511570            7.0
998    1714.919079    1002.480028   -506.945023            4.0
999   -1707.817242    -129.617312   -618.130067            5.0

[1000 rows x 4 columns]

 - - PCA for 3 Continuous Clusters

## Part 4: T-SNE (2 points)
Please use the sklearn's T-SNE class to transform the original data

In [150]:
X_tsne = X
plot_clustering(X_tsne, y, "TSNE PCA", "TSNE")

Data after dimensionality reduction:
          TSNE 0        TSNE 1       TSNE 2 clothing_type
0     -8.389761     1.8965592      5.21573   T-shirt/top
1     2.7244105    0.45147842     11.62055       Trouser
2     0.5054757  -0.060513698  -0.88382107      Pullover
3    -6.5967765     2.7745724   -3.5374696      Pullover
4     3.8851388      3.649186     7.735289         Dress
..          ...           ...          ...           ...
995  0.47319403    -5.8815975   -14.765647    Ankle boot
996  -7.9424114      9.035027   -1.5441602      Pullover
997    6.290246     -7.527712     -4.41744       Sneaker
998  -4.9483004      8.479512   -5.8761554          Coat
999   6.1381984     -7.985258    1.0703132        Sandal

[1000 rows x 4 columns]

 - - TSNE PCA for 3 Discrete Clusters:





Key:  {0: 'T-shirt/top', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat', 5: 'Sandal', 6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle boot'}
Data after dimensionality reduction:
         TSNE 0     TSNE 1    TSNE 2  clothing_type
0   -12.840626  -0.828632 -0.019086            0.0
1    -3.548444 -10.516208 -4.324576            1.0
2     0.352489   2.160808 -1.052118            2.0
3    -1.361492   4.289675  7.169224            2.0
4    -5.055390  -6.657488 -7.585267            3.0
..         ...        ...       ...            ...
995  14.214435  10.344587  3.923450            9.0
996  -7.857392   3.660775  8.987644            2.0
997  11.387494   0.659400 -2.663807            7.0
998  -6.569464   7.894608  6.221069            4.0
999   8.027203  -3.522619 -4.324070            5.0

[1000 rows x 4 columns]

 - - TSNE PCA for 3 Continuous Clusters:


## Part 5: Question (1 point)

Please answer in what situation, PCA or T-SNE would be a better option compared with the other. Plase input your answer below.

### T-SNE seems to be the overall better option. Though both algorithms have troubel differentiating dresses vs. pullovers vs. trousers, in both the 3-component and 2-component analyses, T-SNE has an easier time properly distinguishing most of the components, especially bags, sneakers, and ankle-boots.
Note: for discrete clusters both T-SNE and PCA seem to perform perfectly so it is hard to evaluate